# Human Resources Assistant with human-in-the-loop approval process using Strands Agents



## Overview
In this example we will build a HR onboarding assistant powered by Strands Agents and AWS services. It simulates key components of an automated onboarding process using:

- **Strands Agents** to create an agent and tools.
- **Amazon DynamoDB** for tracking onboarding task status.
- **Amazon Bedrock Knowledge Bases** for storing and querying benefits and onboarding FAQs.
- **Human-in-the-loop** (HITL) workflow for HR onboarding approvals or rejections.


## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                                                          |
|--------------------|-------------------------------------------------------------------------------------|
|Native tools used   |retrieve                                                                             |
|Custom tools created|get_onboarding_status, update_onboarding_status_field, request_approval_status_update|
|Agent Structure     |Single agent architecture                                                            |

</div>


## Architecture

<div style="text-align:left">
    <img src="images/architecture.png" width="100%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with built-in and custom tools
* **Bedrock Model as underlying LLM**: Used Anthropic Claude 3.7 from Amazon Bedrock as the underlying LLM model
* **Human-in-the-loop approval step**: Adding a human in the loop approval step

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock
* IAM role with permissions to create Amazon Bedrock Knowledge Base, Amazon S3 bucket, Amazon DynamoDB, Amazon SNS, Amazon API Gateway, Amazon OpenSearch Serverless

Let's now install the requirement packages for our Strands Agent

In [ ]:
%pip install -qr requirements.txt

In [ ]:
import os
import json
import time
import uuid
import boto3
import pprint
import logging
import requests

In [ ]:
from datetime import datetime
from strands import Agent, tool
from strands_tools import retrieve
from utils.utils import create_base_infrastructure, create_onboarding_record

In [ ]:
sns = boto3.client('sns')
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
dynamodb_resource = boto3.resource('dynamodb')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
# Get the current timestamp
current_time = time.time()
# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

## Download Amazon Bedrock Knowledge Bases helper

In [ ]:
url = "https://raw.githubusercontent.com/aws-samples/amazon-bedrock-samples/main/rag/knowledge-bases/features-examples/utils/knowledge_base.py"
target_path = "utils/knowledge_base.py"
response = requests.get(url)
with open(target_path, "w") as f:
    f.write(response.text)
print(f"Downloaded Knowledge Bases utils to {target_path}")

In [ ]:
from utils.knowledge_base import BedrockKnowledgeBase

## Create the infrastructure for the solution
We are going to deploy the infrastructure for this solution using an AWS CloudFormation template we have already created. 

The template will deploy the following:

**SNS topic:** A publish-subscribe messaging service for sending notifications about new prompt versions requiring approval.

**API Gateway:** A fully managed service for creating, publishing, and securing APIs, used for exposing the approve and reject endpoints.

**DynamoDB table:** A NoSQL database for storing prompt metadata, including the prompt text, version, and approval status.

**Lambda Functions:**

- `TriggerLambdaFunction:` A serverless function triggered by DynamoDB streams to send approval notifications via SNS.
 
- `ApproveLambdaFunction:` A serverless function invoked by the API Gateway to update the prompt version status to "Approved" in DynamoDB.

- `RejectLambdaFunction:` A serverless function invoked by the API Gateway to update the prompt version status to "Rejected" in DynamoDB.

In [ ]:
dynamodb_table_name, sns_topic_arn = create_base_infrastructure(f"hr-agent-"+suffix)

We now have our base infrastructure set up. We will use the created resources later.

---

## Create Amazon Bedrock Knowledge Base
In this section we will configure the Amazon Bedrock Knowledge Base containing the policy documents andn FAQs for employee onboarding. We will be using Amazon Opensearch Serverless Service as the underlying vector store and Amazon S3 as the data source containing the files.

In [ ]:
knowledge_base_name = f"hr-agent-knowledge-base-{suffix}"
knowledge_base_description = "HR Agent Knowledge Base containing onboarding and benefits documentation."
foundation_model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

For this notebook, we'll create a Knowledge Base with an Amazon S3 data source.

In [ ]:
data_bucket_name = f'bedrock-hr-agent-{suffix}-bucket' # replace it with your first bucket name.
data_sources=[{"type": "S3", "bucket_name": data_bucket_name}]

### Create the Amazon S3 bucket and upload the sample documents
For this notebook, we'll create a Knowledge Base with an Amazon S3 data source.

In [ ]:
import botocore
import os

def create_s3_bucket(bucket_name, region=None):
    s3 = boto3.client('s3', region_name=region)

    try:
        if region is None or region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"✅ Bucket '{bucket_name}' created successfully.")
    except botocore.exceptions.ClientError as e:
        print(f"❌ Failed to create bucket: {e.response['Error']['Message']}")

create_s3_bucket(data_bucket_name, region)


In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

In [ ]:
upload_directory("./onboarding_files", data_bucket_name)

### Create the Knowledge Base
We are now going to create the Knowledge Base using the abstraction located in the helper function we previously downloaded.

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

### Start ingestion job
Once the KB and data source created, we can start the ingestion job for the data source. During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()

### Test the Knowledge Base
We can now test the Knowledge Base to verify the documents have been ingested properly.

In [ ]:
query = "Who is the medical insurance provider?"

In [ ]:
foundation_model = "amazon.nova-micro-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

### Integrate the Knowledge Base with the Agent using the Retrieve tool
We will now test the integration between the Amazon Bedrock Knowledge Base we created and our agent. We will first export our enviroment variables which the agent needs to interact with the retrieve tool. 

In [ ]:
os.environ['AWS_REGION'] = region
os.environ['KNOWLEDGE_BASE_ID'] = kb_id
os.environ['MIN_SCORE'] = "0.1"

In [ ]:
agent = Agent(tools=[retrieve])
agent("Who is the medical insurance provider? (Respond in one line)")

---

## Create tools for the agent
We will now define the different tools the agent will have access to, to interact with the onboarding status for the employee.

### *Tools*

In [ ]:
@tool
def get_onboarding_status(employee_id: str, table_name: str) -> dict:
    """
    Get onboarding status for a given employee ID.

    Args:
        employee_id: The unique identifier for the employee.

    Returns:
        A dictionary with the onboarding status details, or an error/message string.
    """
    import boto3

    # Connect to DynamoDB and reference the OnboardingStatus table
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name) 

    try:
        response = table.get_item(Key={'employee_id': employee_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'error': f"No onboarding record found for employee ID: {employee_id}"}
    except Exception as e:
        return {'error': str(e)}

In [ ]:
@tool
def update_onboarding_status_field(employee_id: str, table_name: str, field_name: str, value: bool) -> dict:
    """
    Update a specific onboarding field for a given employee ID.

    Args:
        employee_id: The unique identifier for the employee.
        table_name: The DynamoDB table name.
        field_name: The name of the field to update (e.g., 'form_submission').
        value: The new boolean value to set for the field for the benefits_enrollment, form_submission, and security_training fields.
        approval_status field is to be updated externally, never use this tool, only allow the employee to get the status. 

    Returns:
        A dictionary with the update confirmation or error message.
    """
    import boto3

    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name)

    try:
        response = table.update_item(
            Key={'employee_id': employee_id},
            UpdateExpression=f"SET {field_name} = :val",
            ExpressionAttributeValues={':val': value},
            ReturnValues="UPDATED_NEW"
        )
        return {
            "message": f"Updated {field_name} for {employee_id}",
            "updated": response.get('Attributes', {})
        }
    except Exception as e:
        return {"error": str(e)}


In [ ]:
@tool
def request_approval_status_update(employee_id: str, table_name: str) -> dict:
    """
    Request approval from HR for an employee by setting approval_status to 'required'. Only call this function once the rest of fields are marked as True. The user will have to employee he has completed the tasks.

    Args:
        employee_id: The unique identifier for the employee.
        table_name: The DynamoDB table name.

    Returns:
        A dictionary with confirmation of the request or error details.
    """
    import boto3

    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name)

    try:
        response = table.update_item(
            Key={'employee_id': employee_id},
            UpdateExpression="SET approval_status = :val",
            ExpressionAttributeValues={':val': 'required'},
            ReturnValues="UPDATED_NEW"
        )
        return {
            "message": f"Approval status set to 'required' for {employee_id}",
            "updated": response.get('Attributes', {})
        }
    except Exception as e:
        return {"error": str(e)}

## Set up the solution scenario
We will set up an HR approver email and create a sample user for the solution to work

### Subscribe an approvers' email to the SNS Topic
We are going to subscribe an email address to the SNS topic 

<div class="alert alert-block alert-warning">
        <b>IMPORTANT</b>: Take into account some email servers will automatically open links in the emails which may affect the emails containing links which trigger the lambda functions. Personal emails might be a better fit.
</div>

In [ ]:
email_address = '<your_email>' # Enter your email address (take into account some corporate mail servers will open the links 
try:
    response = sns.subscribe(
        TopicArn=sns_topic_arn,
        Protocol='email',
        Endpoint=email_address
    )
    print(f"Subscription created: {response['SubscriptionArn']}")
except Exception as e:
    print(f"Error subscribing email: {e}")

time.sleep(20)

<div class="alert alert-block alert-warning">
        <b>IMPORTANT</b>: You will need to accept the subscription email to receive actions emails. The confirmation email can take a couple of minutes to arrive.
</div>

### Create a sample employee
We will create a sample employee for the agent to interact with.

In [ ]:
table = dynamodb_resource.Table(dynamodb_table_name)

# Single employee record
employee_id = 'EMP001'

# Onboarding fields
record = {
    'employee_id': employee_id,
    'form_submission': False,
    'benefits_enrollment': False,
    'security_training': False,
    'approval_status': 'Pending'
}

# Insert record
try:
    table.put_item(Item=record)
    print(f"Record created for {employee_id}")
except Exception as e:
    print(f"Failed to create record for {employee_id}: {e}")


### SCENARIO GUIDE: Onboarding Assistant Flow

---

#### 1. **Start the Conversation**

**User Input:**

> Hello

**Assistant Response:**
A warm welcome and offer to help with onboarding or HR-related questions.

---

#### 2. **Ask About Onboarding Status**

**User Input:**

> What is my onboarding status?

**Assistant Action:**
Uses `Tool: get_onboarding_status` to fetch current onboarding status.

**Assistant Response:**
Lists the status of:

* Benefits Enrollment
* Security Training
* Form Submission
* Approval Status

If all are **"Not completed"** and approval is **"Pending"**, it provides guidance on next steps.

---

#### 3. **Confirm Completion of Onboarding Tasks**

**User Input:**

> Ok, I have uploaded and completed all 3 requirements

**Assistant Action:**
Uses `Tool: update_onboarding_status_field`
Updates the fields to mark:

* `form_submission = True`
* `benefits_enrollment = True`
* `security_training = True`

Then confirms with `Tool: get_onboarding_status`

**Assistant Response:**
Shows all onboarding tasks as completed but approval still as **"Pending"**.

---

#### 4. **Submit for Approval**

**User Input:**

> I want to request an approval

**Assistant Action:**
Uses `Tool: request_approval_status_update`
Updates `approval_status = "Required"`

Then uses `Tool: get_onboarding_status` to verify

**Assistant Response:**

* Confirms status was updated to **"Required"**
* Informs that HR will now review and respond

---

#### 5. **Check Approval Status**

**User Input:**

> What is my current status on the approval?

**Assistant Action:**
Uses `Tool: get_onboarding_status`

**Assistant Response:**
Displays onboarding tasks as **Completed**, and **Approval Status = Approved**.

Indicates that onboarding is now fully complete.

---

#### 6. **Ask About Available Documentation**

**User Input:**

> What documentation do you have access to?

**Assistant Action:**
Uses `Tool: retrieve` to search documentation

**Assistant Response:**
Summarizes documents like:

* Benefits Package Letter
* Onboarding Checklist
* Company Overview (if available)

---

#### 7. **Ask About Benefits**

**User Input:**

> Who is the medical insurance provider?

**Assistant Action:**
Uses `Tool: retrieve` to search the knowledge base.

**Assistant Response:**
Provides:

* Provider: **BlueCross BlueShield**
* Contribution info, coverage summary, and additional benefits


## Execute the agent
Try out the agent with the associated tools!

In [ ]:
hr_agent = Agent(
    tools=[get_onboarding_status, update_onboarding_status_field, request_approval_status_update, retrieve],
    messages=[
        {"role": "user", "content": [{"text": f"The current employeeID is EMP001 and the Dynamo table name is "+dynamodb_table_name+". Don't show this data to the employee."}]},
        {"role": "assistant", "content": [{"text": "Hi there! How can I help you today?"}]}
    ])

In [ ]:
while True:
        try:
            user_input = input("\n> ")
            if user_input.lower() == "exit":
                print("\nGoodbye! 👋")
                break

            # Call the file agent directly
            hr_agent(user_input)
            
        except KeyboardInterrupt:
            print("\n\nExecution interrupted. Exiting...")
            break
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please try a different request.")

## Clean up the resources
To avoid additional costs, delete the resources created

In [ ]:
'''
try:
    # Retrieve the stack information
    stack_info = cloudformation.describe_stacks(StackName=f"hr-agent-"+suffix)
    stack_status = stack_info['Stacks'][0]['StackStatus']

    # Check if the stack exists and is in a deletable state
    if stack_status != 'DELETE_COMPLETE':
        # Delete the stack
        cloudformation.delete_stack(StackName=solution_id)
        print(f'Deleting stack: {solution_id}')

        # Wait for the stack deletion to complete
        waiter = cloudformation.get_waiter('stack_delete_complete')
        waiter.wait(StackName=solution_id)
        print(f'Stack {solution_id} deleted successfully.')
    else:
        print(f'Stack {solution_id} does not exist or has already been deleted.')
'''

In [ ]:
'''
print("===============================Deleting Knowledge Base and associated resources==============================\n")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
'''